In [39]:
import tensorflow as tf

print(tf.__version__)

2.3.0


In [40]:
!pip install -q tensorflow-datasets

In [41]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

In [42]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [58]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []



In [59]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [60]:
for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf-8'))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf-8'))
  testing_labels.append(l.numpy())

In [61]:
print(training_sentences[2])

Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.


In [62]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [63]:
training_padded.shape

(25000, 120)

In [64]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [65]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(6, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [66]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.fit(training_padded, 
          training_labels_final, 
          epochs=10, 
          validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4909 - accuracy: 0.7475 - val_loss: 0.8744 - val_accuracy: 0.5234
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2441 - accuracy: 0.9036 - val_loss: 1.0568 - val_accuracy: 0.5207
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1016 - accuracy: 0.9733 - val_loss: 1.3255 - val_accuracy: 0.5174
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0268 - accuracy: 0.9963 - val_loss: 1.6432 - val_accuracy: 0.5150
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0067 - accuracy: 0.9994 - val_loss: 1.9016 - val_accuracy: 0.5152
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 2.1104 - val_accuracy: 0.5162
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.2061e-04 - accuracy: 1.0000 - val_loss: 2.2786 - val_accuracy: 0.5160
Ep

In [75]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

? ? ? ? ? ? ? ? i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what

In [69]:
fpadded[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,   12,    7,
          2,  241,    5,   20,   16,    4, 8776, 2707, 2653,   52,    2,
        358,    5,    2,  180,   68,  138, 1401,   17,   92,  203,  968,
         15,   23,    1,   81,    4,  192, 3109, 3037,    3,    1,   16,
          4,  376,    5,  632,  387,  352,   37, 6355,    3, 5559, 1928,
         15,  208, 8596, 3433,    2,  112,  365,   48,   24,   55,    1,
          6, 1657,   55, 1818, 4312,   41,    4, 2263,    3, 1908,    1,
        141,  160,  780,  111,   31,   92,  116,    4,  221,   20,    9,
        173,  279,    3,   29,   13, 1012,    2, 2814,   10, 1985],
      dtype=int32)

In [70]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [71]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [73]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
sentence = "I really think this ia amazing. honest."

sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 1, 478, 1200]]
